In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd

data1 = pd.read_csv("/kaggle/input/iscx-tor-nontor-2017-csvs/Scenario-A-merged_5s.csv")
data1.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR


In [3]:
data1.shape

(84194, 29)

In [4]:
data2 = pd.read_csv("/kaggle/input/iscx-tor-nontor-2017-csvs/Scenario-B-merged_5s.csv")

In [5]:
data2.shape

(14508, 29)

In [6]:
combined = pd.concat([data1, data2])
combined.shape

(98702, 29)

In [7]:
combined.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR


In [8]:
combined.columns = combined.columns.str.strip(' ')
combined.columns.tolist()

['Source IP',
 'Source Port',
 'Destination IP',
 'Destination Port',
 'Protocol',
 'Flow Duration',
 'Flow Bytes/s',
 'Flow Packets/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Flow IAT Min',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Fwd IAT Min',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Active Mean',
 'Active Std',
 'Active Max',
 'Active Min',
 'Idle Mean',
 'Idle Std',
 'Idle Max',
 'Idle Min',
 'label']

In [9]:
data = combined[['Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Flow Packets/s']]

In [10]:
data.head()

,Source IP,Source Port,Destination IP,Destination Port,Flow Packets/s
0,10.0.2.15,53913,216.58.208.46,80,4597.701149
1,10.0.2.15,53913,216.58.208.46,80,7722.007722
2,10.0.2.15,53913,216.58.208.46,80,2244.668911
3,10.0.2.15,53913,216.58.208.46,80,1862.197393
4,10.0.2.15,53913,216.58.208.46,80,6349.206349


In [11]:
data.shape

(98702, 5)

In [12]:
data.to_csv('fivetupleflow.csv')

## Finetuning

In [13]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

In [14]:
#convert the flows to text- maybe one sentence for one row? 

# first we find a way to process the csv to merge the rows together
def preprocess_data(csv_path):
    df = pd.read_csv(csv_path)
    df["text"] = df.apply(lambda row: f"SRC_IP: {row['Source IP']}, SRC_PORT: {row['Source Port']}, "
                                    f"DST_IP: {row['Destination IP']}, DST_PORT: {row['Destination Port']}, "
                                    f"PACKETS/s: {row['Flow Packets/s']}", axis=1)
    return df[["text"]]

In [15]:
csv= '/kaggle/working/fivetupleflow.csv'
df = preprocess_data(csv)
df.head()
# so now we have all the columns merged into one for each row
# each row is like one sentence

,text
0,"SRC_IP: 10.0.2.15, SRC_PORT: 53913, DST_IP: 21..."
1,"SRC_IP: 10.0.2.15, SRC_PORT: 53913, DST_IP: 21..."
2,"SRC_IP: 10.0.2.15, SRC_PORT: 53913, DST_IP: 21..."
3,"SRC_IP: 10.0.2.15, SRC_PORT: 53913, DST_IP: 21..."
4,"SRC_IP: 10.0.2.15, SRC_PORT: 53913, DST_IP: 21..."


In [16]:
# okay so next we'd have to convert this to a hugginface dataset so that we can finetune a basic bert model
def convertDataset(df):
    return Dataset.from_pandas(df)

In [17]:
dataset = convertDataset(df)

In [18]:
dataset

Dataset({
    features: ['text'],
    num_rows: 98702
})

In [19]:
from datasets import DatasetDict

In [20]:
# next we will have to tokenise 
# we'll be initializing the tokenizer for the bert model using the hugging face transformers library
# we're basically loading the pretrained tokenizer that's associated with the model that we're trying to finetune
# this tokenizer uses the wordpiece algorithm to split text into smalelr units (tokens) 
# it maps the tokens to a unique integer IS based off the model's vocab

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [21]:
def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

In [22]:
def finetune_bert(dataset):
    tokenized_dataset = dataset.map(tokenize, batched=True)  # this applies the tokenization we just discussed
    
    # gotta split the dataset with a 10% eval so that the embeddings are alright
    dataset_split = tokenized_dataset.train_test_split(test_size=0.1)  # 90% train, 10% eval
    
    model = BertForMaskedLM.from_pretrained("bert-base-uncased")  # here we load the generic bert model SPECIFICALLY for masked language modelling
    
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)  
    # this just prepares batches of data with dynamic masking
    # it randomly masks 15% of the tokens per batch and also uses tokenizer to handle the padding and masking

    training_args = TrainingArguments( 
        output_dir="./bert_finetuned",  # this is where the training outputs will go
        evaluation_strategy="epoch",  # we're evaluating the model after every training epoch
        save_strategy="epoch",  # save model ka checkpoint after every epoch
        per_device_train_batch_size=8,  # 8 samples per core during training
        per_device_eval_batch_size=8,  # 8 samples per core during evaluation
        num_train_epochs=3,  # trains the model for this many passes through the dataset
        weight_decay=0.01,  # l2 regularization to prevent overfitting
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_split["train"],
        eval_dataset=dataset_split["test"],  # Added this!
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    trainer.train()  # Start training

    model.save_pretrained("./bert_finetuned")
    tokenizer.save_pretrained("./bert_finetuned")

In [ ]:
finetune_bert(dataset)

Map:   0%|          | 0/98702 [00:00<?, ? examples/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

<IPython.core.display.Javascript object>

In [ ]:
# now we just test the generated metrics
def generate_embeddings(texts, model_path="./bert_finetuned"):
    model = BertForMaskedLM.from_pretrained(model_path)
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model.eval()

    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model.bert(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # CLS token embedding
    return embeddings

# embeddings = generate_embeddings(["SRC_IP: 10.0.2.15, SRC_PORT: 53913, DST_IP: 216.58.208.46, DST_PORT: 80, PACKETS/s: 4597.701149"])
